Homework 6

4.
suppose that 50 people are given a placebo and 50 are given a new treatment. 30 placebo patients show improvement while 40 treated patients show improvement. Let $\tau = p_2-p_1$ where $p_2$ is the probability of improving under treatment and $p_1$ is the probability of improving under placebo

(b) Find the standard error and 90 percent confidence interval using the parametric bootstrap

In [23]:
import numpy as np
from scipy.stats import binom
from scipy.stats import norm

In [24]:
x1 = 30
x2 = 40
n1 = 50
n2 = 50
p1_hat = x1/n1
p2_hat = x2/n2
tau_hat = p1_hat - p2_hat
B = 10**4
alpha = 0.1

In [25]:
placebo_imp = binom.rvs(n1, p1_hat, size = B)
treatment_imp = binom.rvs(n2, p2_hat, size = B)
tau_boot = treatment_imp/n2 - placebo_imp/n1
tau_hat = np.mean(tau_boot)

lower = np.quantile(tau_boot, 0.05)
upper = np.quantile(tau_boot, 0.95)
print("standard error is: ", se)
print("90 percent confidence interval is: ( {:.5f} , {:.5f})".format(lower, upper) )


NameError: name 'se' is not defined

(c) Use the prior $f\left( p_1, p_2 \right) =1$. Use simulation to find the posterior mean and posterior 90 percent interval for $\tau$

In [4]:
from scipy.stats import beta

$X_1, X_2, \dots X_n \sim Bernoulli(p)$ and for the prior take the uniform distribution $f(p)=1$ on the domain $[0,1]$

let $s=\sum_{i=1}^{n} x_i$, the posterior is $f(p|x^n) \propto Beta(\alpha+1, \beta+n+1)$, where $\alpha = 1, \beta =1$

$p_1|X1 \sim Beta(X_1+1, n1-X_1+1)$,
$p_2|X2 \sim Beta(X_2+1, n2-X_2+1)$

In [5]:
x1 = 30
x2 = 40
n1 = 50
n2 = 50
B = 10**4
alpha = 0.1

In [6]:
p1_x1 = beta.rvs(x1+1, n1-x1+1, size = B)
p2_x2 = beta.rvs(x2+1, n2-x2+1, size = B)
tau_boot = p2_x2 - p1_x1
tau_posterior = np.mean(tau_boot)

lower = np.quantile(tau_posterior, 0.05)
upper = np.quantile(tau_posterior, 0.95)
print("posterior mean is: ", tau_posterior)
print("90 percent confidence interval for tau is: ( {:.5f} , {:.5f})".format(lower, upper) )

posterior mean is:  0.1931764219942001
90 percent confidence interval for tau is: ( 0.04879 , 0.33756)


(e)

In [7]:
p1_x1 = beta.rvs(x1+1, n1-x1+1, size = B)
p2_x2 = beta.rvs(x2+1, n2-x2+1, size = B)
psi_boot = np.log(p1_x1) - np.log(1-p1_x1)-np.log(p2_x2)+ np.log(1-p2_x2)
psi_posterior = np.mean(psi_boot)
se_psi = np.std(psi_boot)
lower = psi_posterior - se_psi * z_10
upper = psi_posterior + se_psi * z_10
print("posterior mean is: ", psi_posterior)
print("90 percent confidence interval for psi is: ( {:.5f} , {:.5f})".format(lower, upper) )



posterior mean is:  -0.9532188339383978
90 percent confidence interval for psi is: ( -1.68584 , -0.22060)


Problem 8

Compare $P(H_0|X=x)$ and p-value of the Wald test. Do the comparison numerically for a variety of values of $x$ and $b$

${\displaystyle P(X=x|H_1)= \frac{e^{-\frac{x^2}{2}}}{2\pi \cdot b} {\displaystyle \int_{-\infty}^{\infty}}e^{-\frac{b^2+1}{2b^2}\mu ^2 + x\mu}d\mu = \frac{e^{-\frac{x^2}{2}+\frac{b^2x^2}{2b^2+2}}}{2\pi b}\sqrt{\frac{2b^2\pi}{b^2+1}}}$



${\displaystyle P(H_0|X=x) = \frac{\frac{1}{\sqrt{2\pi}}e^{-\frac{x^2}{2}}}{\frac{1}{\sqrt{2\pi}}e^{-\frac{x^2}{2}} + \frac{e^{-\frac{x^2}{2}+\frac{b^2x^2}{2b^2+2}}}{2\pi b}\sqrt{\frac{2b^2\pi}{b^2+1}}}}$

In [5]:
from scipy.stats import uniform
from scipy.integrate import quad
import numpy as np
from scipy.stats import norm


In [27]:
z_5 = norm.ppf(1- 0.05/2)

In [18]:
def pvalue_ind(x):
    if abs((x))> z_5:
        return 1 ## reject H0
    else:
        return 0 ## not reject H0
    

In [19]:
def integral(b, x):
    return np.exp(-x**2/2 + (b**2 * x**2)/(2*b**2+2))* np.sqrt(2*b**2* np.pi/(b**2+1)) / (2 * np.pi*b)

In [20]:
b = np.random.uniform(0.5, 2, 60)
x = np.zeros((1, 60))
for i in range(60):
    mu = np.random.normal(0,b[i],1)[0]
    x[0][i] =  np.random.normal(mu, 1, 1)
pair = np.transpose(np.array([b, x], dtype=object))

In [21]:
int_answer= np.zeros((1,60))
p_val_ind = np.zeros((1,60))
for i in range(60):
    x_current = pair[1][0][i]
    b_current = pair[0][i]
    p_val_ind [0][i]= pvalue_ind(x_current)
    int_answer[0][i] = integral(b_current, x_current)
p_val = np.mean(p_val_ind)
print("the p_value is: ", p_val)

the p_value is:  0.18333333333333332


In [22]:
x = pair[1][0]
numerator = np.empty([1,60])
for i in range(60):
    numerator[0][i] = (1/np.sqrt(2*np.pi)) * np.exp(-x[i]**2/2)
prob_H0 = numerator/(numerator + int_answer)
p_H0_mean = np.mean(prob_H0)
print("the posterior probability of H0 is: ", p_H0_mean)

the posterior probability of H0 is:  0.4948957897959989


repeat the problem using a sample size n=100

$ {\displaystyle P(H_0| (x_1, x_2, \dots x_n ))= \frac{(\frac{1}{\sqrt{2\pi}})^{n} \cdot e^{-\frac{x_1^2 + x_2^2 + \dots x_n^2 }{2}}}{(\frac{1}{\sqrt{2\pi}})^{n} \cdot e^{-\frac{x_1^2 + x_2^2 + \dots x_n^2 }{2}} + {\displaystyle \prod_{i=1}^{n}}\frac{e^{-\frac{x^2_{ni}}{2}}}{2\pi b}\int_{-\infty}^{\infty}e^{-\frac{b^2+1}{2b^2}\mu^2+x_{ni}\mu}d\mu}}$



${\displaystyle p-value = P(\lvert \frac{\bar{X_n}}{\sqrt{n}}\rvert > z_{\alpha/2})}$

In [23]:
def pvalue_ind_n(x, n):
    if abs(np.mean(x)/(np.sqrt(n)))> z_5:
        return 1 ## reject H0
    else:
        return 0 ## not reject H0

In [24]:
def integral(b, x):
    return np.exp(-x**2/2 + (b**2 * x**2)/(2*b**2+2))* np.sqrt(2*b**2* np.pi/(b**2+1)) / (2 * np.pi*b)

In [25]:
n = 200
z_5 = norm.ppf(1- 0.05/2)
b = np.random.uniform(0.5, 2, 60)
x = np.empty([60, n])
for i in range(60):
    mu = np.random.normal(0,b[i],1)[0]
    x[i] = np.random.normal(mu, 1, n)

In [26]:

p_val_ind_n = np.zeros((1,60))
prod = np.zeros((1, 60))

for i in range(60):
    need_product = np.zeros((1, n))
    b_current = b[i]
    for j in range(n):
        x_current = x[i][j]
        need_product[0][j] = integral(b_current, x_current)
    prod[0][i] = np.prod(need_product)
    p_val_ind_n[0][i]= pvalue_ind_n(x[i], n)




    

In [27]:
numerator = np.zeros((1, 60))
for i in range(60):
    sum_square = np.dot(x[i], x[i])
    exp = (-1/2)* sum_square
    numerator[0][i]= (1/np.sqrt(2* np.pi))**n * np.exp(exp)
    

In [28]:
p_value_n = np.mean(p_val_ind_n)
p_H0_n = numerator / (numerator + prod)
p_H0_n_mean = np.mean(p_H0_n)   
    
    
print("the posterior probability of H0 is: ", p_H0_n_mean)
print("the p_value is: ", p_value_n)


the posterior probability of H0 is:  0.4549032931621652
the p_value is:  0.0


Notice that the posterior probability of H0 can be large even when the p-value is small, especially when n is large.

In [29]:
p_H0_n

array([[4.27833177e-052, 3.87002330e-001, 2.26533809e-056,
        8.79755832e-011, 0.00000000e+000, 9.99833649e-001,
        2.66069814e-044, 1.00000000e+000, 3.20483371e-010,
        1.00000000e+000, 3.34812869e-046, 9.98003505e-001,
        3.88390868e-006, 0.00000000e+000, 2.15308611e-003,
        9.99996269e-001, 2.72331248e-052, 6.80665407e-012,
        9.97763431e-001, 2.75137666e-006, 1.00000000e+000,
        9.96973218e-001, 9.99995012e-001, 1.14601473e-105,
        9.94135120e-001, 1.00000000e+000, 4.80957061e-041,
        1.39464974e-088, 0.00000000e+000, 9.99999999e-001,
        2.59321093e-076, 9.99999997e-001, 9.99999909e-001,
        1.00000000e+000, 7.08566917e-036, 9.99999699e-001,
        8.00235606e-056, 3.35793998e-008, 9.99796644e-001,
        1.00000000e+000, 7.19843990e-067, 1.00000000e+000,
        9.99543465e-001, 4.02340880e-049, 0.00000000e+000,
        1.94831393e-062, 8.00840776e-005, 9.99999994e-001,
        2.02651555e-003, 0.00000000e+000, 9.16891427e-00